# LORA Fine-tuning

In [1]:
from datasets import *
import datasets

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np 

In [2]:

dataset = load_dataset('xTRam1/safe-guard-prompt-injection')
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8236
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2060
    })
})

### model

In [3]:
model_checkpoint = "distilbert-base-uncased"
model_output = "distilbert-base-uncased-lora-text-classification-safeguard-promptinjection"

# define label maps
id2label = {0: "Good", 1: "Prompt Injection"}
label2id = {"Good": 0, "Prompt Injection": 1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id, trust_remote_code=True)

c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### preprocess data

In [5]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [6]:
# create tokenize function
def tokenize(batch):

    #tokenize and truncate text
    tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
    print(batch['label'])
    return tokens


# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8236
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2060
    })
})

In [7]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### evaluation

In [8]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

### Apply untrained model to text

In [9]:
# define list of examples
text_list = ["jailbreak [keyword] [TARGETLANGUAGE] [PROMPT]"]
print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    #print(inputs)
    # compute logits
    logits = model(inputs).logits
    #print(logits)
    # convert logits to label
    predictions = torch.argmax(logits)
    #print(predictions)

    print(id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
Prompt Injection


### Train model

In [10]:

peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=16,
                        lora_alpha=32,
                        lora_dropout=0.05,
                        target_modules=["q_lin", "k_lin", "v_lin"])

In [11]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=16, target_modules={'k_lin', 'q_lin', 'v_lin'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [12]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,034,498 || all params: 67,989,508 || trainable%: 1.5216


In [13]:
# hyperparameters
lr = 1e-3
batch_size = 2
num_epochs = 5

In [14]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_output,
    learning_rate=lr,
    auto_find_batch_size=True,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [15]:
# show metrics for f1 and auprc
from sklearn.metrics import f1_score, precision_recall_curve, auc

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    precision, recall, _ = precision_recall_curve(labels, preds)
    auprc = auc(recall, precision)
    return {
    'f1': f1,
    'auprc': auprc
    }

In [16]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics
)

In [17]:
# train model
trainer.train()

  0%|          | 0/5150 [00:00<?, ?it/s]

{'loss': 0.0973, 'grad_norm': 0.013094925321638584, 'learning_rate': 0.0009029126213592234, 'epoch': 0.49}
{'loss': 0.0598, 'grad_norm': 14.282336235046387, 'learning_rate': 0.0008058252427184466, 'epoch': 0.97}


  0%|          | 0/258 [00:00<?, ?it/s]

{'eval_loss': 0.022046023979783058, 'eval_f1': 0.9961181094662183, 'eval_auprc': 0.9945861072038927, 'eval_runtime': 29.7976, 'eval_samples_per_second': 69.133, 'eval_steps_per_second': 8.658, 'epoch': 1.0}


c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

  0%|          | 0/258 [00:00<?, ?it/s]

{'eval_loss': 0.019994083791971207, 'eval_f1': 0.9970849466177308, 'eval_auprc': 0.9965934445788063, 'eval_runtime': 29.4375, 'eval_samples_per_second': 69.979, 'eval_steps_per_second': 8.764, 'epoch': 5.0}


c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 1575.9955, 'train_samples_per_second': 26.13, 'train_steps_per_second': 3.268, 'train_loss': 0.03236420396462227, 'epoch': 5.0}


TrainOutput(global_step=5150, training_loss=0.03236420396462227, metrics={'train_runtime': 1575.9955, 'train_samples_per_second': 26.13, 'train_steps_per_second': 3.268, 'total_flos': 5585876604764160.0, 'train_loss': 0.03236420396462227, 'epoch': 5.0})

### Generate prediction

In [18]:
model.to('cpu') # moving to mps for Mac (can alternatively do 'cpu')

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu") # moving to mps for Mac (can alternatively do 'cpu')

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
Prompt Injection


### Optional: push model to hub

In [19]:
# option 1: notebook login
# from huggingface_hub import notebook_login
# notebook_login() # ensure token gives write access

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [20]:
hf_name = 'cyrp' # your hf username or org name
model_id = hf_name + "/" + model_output # you can name the model whatever you want

In [21]:
model.push_to_hub(model_id) # save model

c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/4.14M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cyrp/distilbert-base-uncased-lora-text-classification-safeguard-promptinjection/commit/c2e4cde166bbd0ee8c891348b5516f3d74caa3a4', commit_message='Upload model', commit_description='', oid='c2e4cde166bbd0ee8c891348b5516f3d74caa3a4', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
trainer.push_to_hub(model_output) # save trainer

events.out.tfevents.1727291356.DESKTOP-LM2NI5P.20076.0:   0%|          | 0.00/9.15k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/cyrp/distilbert-base-uncased-lora-text-classification-safeguard-promptinjection/commit/e6e0c4f95fc5a63e2b1c7d9158b0dfb760d00257', commit_message='distilbert-base-uncased-lora-text-classification-safeguard-promptinjection', commit_description='', oid='e6e0c4f95fc5a63e2b1c7d9158b0dfb760d00257', pr_url=None, pr_revision=None, pr_num=None)

### Optional: load peft model

In [23]:
# how to load peft model from hub for inference
# config = PeftConfig.from_pretrained(model_id)
# inference_model = AutoModelForSequenceClassification.from_pretrained(
#     config.base_model_name_or_path, num_labels=2, id2label=id2label, label2id=label2id
# )
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# model = PeftModel.from_pretrained(inference_model, model_id)